# Informed search - the A* algorithm

The depth-first search and breadth-first search considered in the previous lesson are completely *blind* algorithms: they're only concerned whether the currently considered state is a goal state or not. They're unable to distinguish whether a state is easy or hard to reach, or whether it is near or far of the goal. This makes them very inefficient search algorithms. To allievate the issue, we introduce informed search algorithms. The information is given to an algorithm in two ways:

1. By using an *action cost* function $c(s,a)$, which, given a state $s$ and an action $a$ available in this state, returns its cost as a non-negative number.
2. By using a *heuristic* $h(s)$, which, given a state, estimates the lowest cost to reach a goal state from the given state.

Given a sequence of actions $a_1, \ldots, a_n$ and an initial state $s_1$, we can express the *total cost* of reaching the state $s_{n+1}$ by executing the sequence as:
$$ c(s_1, a_1, \ldots, a_{n-1}) = \sum_{i=1}^n c(s_i, a_i) $$
and the *expected cost* of the solution as the sum of the total cost and the estimate cost of reaching the goal from the state $s_{n+1}$
$$ f(s_1, a_1, \ldots, a_n) = c(s_1, a_1, \ldots, a_n) + h(s_{n+1}) $$

The heuristic function is a bit tricky, because we want it to have two properties:
* *Admissibility*: It must never *overestimate* the true cost of reaching the goal. 
* *Consistency*: Let $s$ be a state such that $a$ is an available action in this state and $s'$ is the state reached by executing this action. The heuristic should fulfil triangle inequality, that is, the estimated cost to reach the goal from $s$ should be no greater than the cost of executing the action $a$ + the estimated cost of reaching the goal from the new state.
$$ h(s) \leq c(s, a) + h(s') $$

One can prove that admissibility follows from consistency, but consistency is important only if there are multiple paths to reach the same state (i.e., we are searching in a graph, not in a tree). Otherwise, admissability is sufficient.

Lets extend the class `Problem` from the previous lesson with two new functions `action_cost` and `heuristic`, which correspond to the functions $c(s,a)$ and $h(s)$ described above.

In [1]:
class Problem:
    @property
    def initial_state(self):
        ...
        
    def available_actions(self, state):
        ...        
        
    def do_action(self, state, action):
        ...
        return new_state
    
    def is_goal(self, state) -> bool:
        ...
        
    def action_cost(self, state, action) -> float:
        ...
        
    def heuristic(self, state) -> float:
        ...

To make a concrete example, lets revisit the vacuum world. 

![](aima-images/fig2_2.png)

Below, we assume a very simple model:
* Any action costs 1. This corresponds to searching for the shortest plan.
* The heuristic estimation is the number of fields which are still dirty. 


Lets consider the properties of the heuristic:
* Is is admissible? The heuristic value is equal to the number of 'Suck' actions that are yet to be executed and ignores the spatial aspect (i.e., moving between the rooms), thus never overestimating.
* Is it consistent? As a consequence of a single action the heuristic value can decrease by at most 1 (if the action happens to be 'Suck' and the room is dirty). The cost of any action is 1, so rewriting the triangle inequality we arrive at:
$$ h(s) \leq c(s, a) + h(s') = \begin{cases} 1 + (h(s)-1) & a=\text{'Suck' and the room was dirty} \\ 1 + h(s) & \text{otherwise} \end{cases} $$
* Is it the best we could have? By no means! We could include the spatial aspect.

In [2]:
class VacuumProblem(Problem):
    @property
    def initial_state(self):
        return (0, (True, True))
    
    def available_actions(self, state):
        return ["Left", "Suck", "Right"]
        
    def do_action(self, state, action):
        robot, dirty = state
        if action == "Left":
            return (max(robot-1, 0), dirty)
        elif action == "Suck":
            new_dirty = list(dirty)
            new_dirty[robot] = False
            return (robot, tuple(new_dirty))
        elif action == "Right":
            return (min(robot+1, len(dirty)-1), dirty)        
        raise Exception('Invalid action')
    
    def is_goal(self, state) -> bool:
        return not any(state[1])
    
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return sum(state[1])

## Task 1: Implement the A* algorithm

To implement the A* algorithm you must have a priority queue. Luckily, Python comes with one, so you don't need to implement it by yourself. Then, the algorithm is very simple: 
1. Start with a queue containing a single item - the initial state
2. Repeat until the queue is not empty:
  1. Pick an item with the lowest expected cost
  2. If this is the goal, return the sequence of actions necessary to reach this state
  3. Otherwise, for each available action, create a new entry in the queue corresponding to the state reached after executing the action.
  
Guard the algorithm against infinite loops: if you already visited a state, you don't need to visit it again (if your heuristic is consistent).

In the cell below implement the algorithm in a similar manner as the BFS and DFS in the previous lesson: the sole argument is an object of the class Problem and the function should return a list of actions to achieve a goal state from the initial state.
If it is impossible to reach the goal, return `None`.
Count the number of states visited during the search (e.g., the number of different states in the collection you use to guard against infinite loops) and print in out before returning from the function, it will be useful later on to compare different heuristics.

In [3]:
# import priority queue
from queue import PriorityQueue

In [4]:
def astar(problem: Problem):
    # get initial measurements
    current_state = problem.initial_state
    current_heuristic = problem.heuristic(current_state)
    current_path = []

    # setup priority queue for states to visit
    candidates = PriorityQueue()
    # (F, h, g, state, path)
    candidates.put((current_heuristic, current_heuristic, 0, current_state, current_path))

    # setup dictionary to track the F value
    f_values = {}

    # counter
    iterations = 0

    # main loop of A*
    while candidates:
        iterations += 1

        # get the state with the least F value
        f, h, g, current_state, current_path = candidates.get()

        # stopping condition
        if problem.is_goal(current_state):
            break

        if current_state not in f_values or f < f_values[current_state]:
            # first record or
            # better path to the state found
            f_values[current_state] = f
        else:
            # if the way we get to the state
            # is worse than before - no need
            # to try to update the neighbors
            # - obviously f will be worse
            continue

        # update states
        for action in problem.available_actions(current_state):
            # get next state and cost
            next_state = problem.do_action(current_state, action)
            action_cost = problem.action_cost(current_state, action)
            
            # calculate functions
            next_g = g + action_cost
            next_h = problem.heuristic(next_state)
            next_f = next_g + next_h

            # put the info about the state, state, and path in the queue
            candidates.put((next_f, next_h, next_g, next_state, current_path + [action]))
        

    print(f'Len: {len(current_path)},\tIterations: {iterations}')

    return current_path

Now lets test your code in the vacuum world!

In [5]:
astar(VacuumProblem())

Len: 3,	Iterations: 6


['Suck', 'Right', 'Suck']

## Task 2: Variants of the vacuum world

Now lets consider a different take on the vacuum world in which the heuristic is not admissible and increases as the number of dirty fields decreases.

In [6]:
class VacuumProblem1(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return len(state[1]) - sum(state[1])
    
astar(VacuumProblem1())

Len: 3,	Iterations: 18


['Suck', 'Right', 'Suck']

And another in which heuristic grossly overestimates the cost.

In [7]:
class VacuumProblem2(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return 10 * sum(state[1])
    
astar(VacuumProblem2())

Len: 3,	Iterations: 6


['Suck', 'Right', 'Suck']

**Which of the three heuristic functions (`VacuumProblem`, `VacuumProblem1`, `VacuumProblem2`) is the best? Is it the expected answer given the properties of the heuristics? If not, explain why an unorthodox approach works better.**

*Answer*:

The best are `VacuumProblem` and `VacuumProblem2`. The first is obvious: we used reasonable heuristic.

Given the small domain, we were lucky enough to get best performance and solution also from `VacuumProblem2`. However, that would not be the case in some instances, as the heuristic is neither admissible (overestimates), nor consistent (follows from previous)

The worst is apparently with the worst heuristic - `VacuumProblem1`, could be called anti-heuristic, as the goal state has the maximum value of heuristic, while the initial - zero.

## Task 3: 8-puzzle problem

Recall the 8-puzzle problem from the previous lesson. Reuse your code and implement an extended version assuming that each action costs 1. Propose 3 (at least) admissible heuristics. This time don't change the initial state, your solution should be capable enough to solve this.

![](aima-images/fig3_4.png)

In [8]:
class PuzzleProblem(Problem):
    @property
    def initial_state(self):
        self.goal = (
            (-1, 1, 2),
            (3, 4, 5),
            (6, 7, 8)
        )

        return (
            (1, 1),
            (
                (7, 2, 4),
                (5, -1, 6),
                (8, 3, 1)
            )
        )
    
    def available_actions(self, state):
        actions = []

        # possible actions
        position, _ = state
        if position[0] > 0:
            actions.append('Up')
        
        if position[0] < 2:
            actions.append('Down')

        if position[1] > 0:
            actions.append('Left')

        if position[1] < 2:
            actions.append('Right')
        return actions
        
    def do_action(self, state, action):
        position, square = state
        square = [list(row) for row in square]
        square = list(square)

        move = (0, 0)

        match action:
            case 'Up':
                move = (-1, 0)
            case 'Down':
                move = (1, 0)
            case 'Left':
                move = (0, -1)
            case 'Right':
                move = (0, 1)
        
        new_position = (max(min(position[0] + move[0], 2), 0), max(min(position[1] + move[1], 2), 0))
        square[position[0]][position[1]] = square[new_position[0]][new_position[1]]
        square[new_position[0]][new_position[1]] = -1
                
        square = tuple([tuple(row) for row in square])
        new_state = (new_position, square)

        return new_state
    
    def is_goal(self, state) -> bool:
        return state[1] == self.goal
        
    def action_cost(self, state, action) -> float:
        return 1
        
    def heuristic(self, state) -> float:
        return sum([
            self.goal[i][j] != state[1][i][j] if state[1][i][j] != -1 else False
            for j in range(3)
            for i in range(3)
        ])

**Prove that this heuristic is admissible.**

*Answer:* `PuzzleProblem`

Although this heuristic may seriously underestimate the true cost of reaching goal, it is admissible. This follows from the fact that we change position of only one piece of puzzle at a time (we do not consider empty space) which results in heuristic change by 1 at maximum. Having the step cost equal to 1, there is no way heuristic underestimates distance.

In [13]:
class PuzzleProblem1(PuzzleProblem):
    def heuristic(self, state) -> float:     
        # Propose another heuristic here
        square = state[1]

        answer = 0
        for i in range(3):
            for j in range(3):
                num = square[i][j]
                if num == -1:
                    continue

                # i * 3 + j
                # we have straightforward goal,
                # so we can calculate the desired location
                # for the value in constant time and space
                # by calculating
                goal_y, goal_x = num // 3, num - num // 3 * 3
                
                answer += abs(i - goal_y) 
                answer += abs(j - goal_x)

        return answer

**Prove that this heuristic is admissible.**

*Answer:* `PuzzleProblem1`

This heuristic provides more information about the state. Admissibility is based on the following thesis: again, __only one piece is moved at a time.__

For every piece we calculate the minimum amount of steps to the desired position regardless of other pieces. With one step we change position for only one piece (empty space is not counted), other parts stay at their positions. Thus, on the turn to move a piece, we cannot hinder another pieces to get to their places, thus we will never predict situation that is worse than could be in reality, at least at specific time moment, when we calculate heuristic.

In [10]:
from math import sqrt

In [11]:
class PuzzleProblem2(PuzzleProblem):
    def heuristic(self, state) -> float:
        # Propose yet another heuristic here
        square = state[1]

        answer = 0
        for i in range(3):
            for j in range(3):
                num = square[i][j]
                if num == -1:
                    continue

                # i * 3 + j
                # same rule over here
                goal_y, goal_x = num // 3, num - num // 3 * 3
                
                answer += sqrt((i - goal_y) ** 2 + (j - goal_x) ** 2)

        return answer

**Prove that this heuristic is admissible.**

*Answer:* `PuzzleProblem2`

Logic is about the same. But this calculation of distance gives less insight, comparing to the Manhattan one, which is admissible.

Run your heuristics on the given problem.

In [12]:
plan0 = astar(PuzzleProblem())
plan1 = astar(PuzzleProblem1())
plan2 = astar(PuzzleProblem2())

print("Is plan0==plan1?", plan0 == plan1)
print("Is len(plan0)==len(plan1)?", len(plan0) == len(plan1))
print("Is plan0==plan2?", plan0 == plan2)
print("Is len(plan0)==len(plan2)?", len(plan0) == len(plan2))
print("Is plan1==plan2?", plan1 == plan2)
print("Is len(plan1)==len(plan2)?", len(plan1) == len(plan2))

Len: 26,	Iterations: 58649
Len: 26,	Iterations: 2298
Len: 26,	Iterations: 10856
Is plan0==plan1? False
Is len(plan0)==len(plan1)? True
Is plan0==plan2? True
Is len(plan0)==len(plan2)? True
Is plan1==plan2? False
Is len(plan1)==len(plan2)? True


**Which of the heuristics is the best for this task? Why is that?**

*Answer:*

- Time & iterations used by each differs dramatically, although all of heuristics gave solutions with identical length. 
- For this task, heuristic with Manhattan distance performed the best, as it gave the best understanding of the "distance" of the problem. 
- Both Euclidean and Manhattan gave significant information, Euclidean is not tied to swapping pieces of the puzzle, whereas Manhattan is.
- Counting pieces that stay in the very right position (first heuristic) is better than nothing, but the information gain is the the worst in this case.

The ranking is the following:
1. Manhattan distance of each piece to right position (greatest amount of information, tied states transformation)
2. Euclidean distance of each piece to right position (decent information)
3. All matches with right positions

------------
The pictures and the description of 8-puzzle are from "Artificial Intelligence: A Modern Approach" 3rd ed.